In [1]:
from transformers import BertTokenizer, TFBertModel
import keras.api._v2.keras

/Users/nick/miniforge3/envs/recSysEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### We now want to move on to creating a matrix of users and rated items for the classical methods such as SVD, ALS OLS and others
## Creating a matrix means having articles and users, and to avoid size issues I can just implement a method that allows a dimnension to be specified 
## It does seem however that the matrix expectation is that of a 2d numpy array instead of a dataframe? Will have to check the lecture stuff
## I mean how hard could it be with a dataframe? Now that I try to work on it, its giving tomorrows issue hahaha.

In [2]:
## How can I apply the embeddings? load and then return the dataframe mythink

In [9]:
def create_text_embeddings(dataset):
    """
    Applies pre-trained BERT embeddings to the feature columns with text data for use within clustering.

    Args:
        dataset (pd.DataFrame) : Dataset containing user interactions.

    Returns:
        dataset (pd.DataFrame) : A dataset with embeddings columns.
    """
    # Initialize the bert tokenizer and model from bert base cased.
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    model = TFBertModel.from_pretrained('bert-base-cased')

    # Create a dense layer with the embedding dimension to process embeddings.
    embedding_dimension = 8
    dense_layer = keras.layers.Dense(embedding_dimension, activation='linear')

    # Define a function to get the embeddings from the models and apply them to the text.
    def get_embeddings(text_1):
        """
        Gets embeddings from pre trained bert model for news information used for clustering.
        """

        # Apply the tokenizer to the text and return it in the tensorflow tensor format. 
        encoded_text = tokenizer(text_1, return_tensors='tf')

        # Get the output from BERT model with the encoded text.
        bert_output = model(encoded_text)

        # Use the pooled output for a single vector representation of the input. 
        pooled_output = bert_output.pooler_output

        # Apply dense layer to project to desired size.
        embedding_vector = list(dense_layer(pooled_output).numpy())
        return embedding_vector

    # Apply the embeddings to the abstract and title columns.
    dataset['abstract_embeddings'] = dataset['abstract'].apply(get_embeddings)
    dataset['title_embeddings'] = dataset['title'].apply(get_embeddings)
    return dataset


In [4]:
import ExploratoryAnalysis.data_processing_modules as dpm
import pandas as pd
import tensorflow as tf
import keras.api._v2.keras as keras
import numpy as np
from sqlalchemy import create_engine

In [5]:
sample_data = pd.read_csv('MIND_small/sample_tf_dataset.csv')
sample_data.drop(columns=['Unnamed: 0'], inplace=True)
sample_data = sample_data[sample_data['abstract'].isna() == False]

In [7]:
subset = sample_data.loc[1:10, ]

In [10]:
data = create_text_embeddings(subset)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
data

,user_id,time_,news_id,category,sub_category,title,abstract,interaction_type,score,abstract_embeddings,title_embeddings
1,U13740,11/11/2019 9:05:58 AM,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...,The Yankees kept hope alive thanks to some imp...,history,1,"[[1.5487633, 1.0190098, -1.1760546, -1.7435807...","[[1.2311747, 1.1580857, -1.0968894, -1.6441334..."
2,U13740,11/11/2019 9:05:58 AM,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,history,1,"[[1.3630767, 1.3074028, -1.0594987, -1.8798711...","[[1.3630767, 1.3074028, -1.0594987, -1.8798711..."
3,U13740,11/11/2019 9:05:58 AM,N45794,news,newscrime,Four flight attendants were arrested in Miami'...,Four American Airlines flight attendants were ...,history,1,"[[1.3427523, 0.9994156, -1.1305358, -1.492023,...","[[1.2866648, 1.0701438, -1.1815537, -1.5371481..."
4,U13740,11/11/2019 9:05:58 AM,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...,"Parting is such sweet sorrow, say the Wolverines.",history,1,"[[1.470107, 1.2682941, -1.3315637, -1.9740347,...","[[1.3234937, 1.0959133, -1.2446544, -1.8129985..."
5,U13740,11/11/2019 9:05:58 AM,N63302,lifestyle,lifestylebuzz,This Wedding Photo of a Canine Best Man Captur...,"When Mark Doublet made his dog, Marley, the be...",history,1,"[[1.3351053, 0.99234486, -1.1616077, -1.547308...","[[1.4223776, 1.3358438, -1.0546858, -1.7920129..."
6,U13740,11/11/2019 9:05:58 AM,N10414,movies,movienews,"Robert Evans, 'Chinatown' Producer and Paramou...","Robert Evans, the Paramount executive who prod...",history,1,"[[1.4915154, 1.0339155, -0.8287275, -1.2396467...","[[1.354709, 1.1848016, -1.0151962, -1.7490983,..."
7,U13740,11/11/2019 9:05:58 AM,N19347,news,newspolitics,Former US Senator Kay Hagan dead at 66,"Former U.S. Sen. Kay Hagan, a one-time Capitol...",history,1,"[[1.3786186, 1.0445164, -1.0278101, -1.4561397...","[[1.1155573, 1.2894244, -1.0174193, -1.6100097..."
8,U13740,11/11/2019 9:05:58 AM,N31801,news,newspolitics,Joe Biden reportedly denied Communion at a Sou...,Joe Biden has a complicated history with the C...,history,1,"[[1.4460843, 0.95543987, -1.1360129, -1.306054...","[[1.2764208, 1.257126, -0.7476753, -0.9903929,..."
9,U13740,11/11/2019 9:05:58 AM,N55689,sports,football_nfl,"Charles Rogers, former Michigan State football...","Charles Rogers, the former Michigan State foot...",impression,0,"[[1.3354195, 1.12606, -1.1411066, -1.5229899, ...","[[1.3344136, 1.2568424, -1.0704972, -1.4981291..."
10,U13740,11/11/2019 9:05:58 AM,N35729,news,newsus,Porsche launches into second story of New Jers...,The Porsche went airborne off a median in Toms...,impression,0,"[[1.3749304, 0.92098105, -0.98385656, -1.32200...","[[1.2780826, 0.92301315, -1.0932304, -1.584886..."


In [7]:
data.dtypes

user_id                object
time_                  object
news_id                object
category               object
sub_category           object
title                  object
abstract               object
interaction_type       object
score                   int64
abstract_embeddings    object
title_embeddings       object
dtype: object

In [4]:
# Starting off we need to load in the dataset and turn it into the form we want
sample_data = pd.read_csv('MIND_small/sample_tf_dataset.csv')
sample_data.drop(columns=['Unnamed: 0'], inplace=True)
sample_data = sample_data[sample_data['abstract'].isna() == False]
AUTOTUNE = tf.data.AUTOTUNE
tf_data_obj = tf.data.Dataset.from_tensor_slices(dict(sample_data))
train_ds = tf_data_obj.cache().prefetch(buffer_size=AUTOTUNE)
text_ds = train_ds.map(lambda x: x['abstract'])

In [ ]:
# Then we need to create a text vectorization layer to handle preprocessing


In [ ]:
test_frame.rename(columns={'time':'time_'}, inplace=True)
test_frame.to_csv('MIND_small/sample_tf_dataset.csv')

def create_connection():
    """
    Creates a connection to the database by taking in a user password.
    """
    database_name = 'user_info'
    connection_user = 'admin'
    try:
        connection_password = input("Enter Password: ")
        connection_host = 'svc-57117697-8d22-448b-8b96-d3b2a46d0970-dml.aws-virginia-6.svc.singlestore.com'
        connection_port = '3306'
        connection_url = f"mysql+pymysql://{connection_user}:{connection_password}@{connection_host}:{connection_port}/{database_name}"
        db_connection = create_engine(connection_url)
        return db_connection
    except Exception as e:
        print(f"Exception {e} occured, try again.\n")
        return create_connection()


def push_to_DB(news : pd.DataFrame, behaviors : pd.DataFrame) -> None:
    """ 
    Pushes data to the SingleStore database for usage in modelling within docker containers and virtual machines.

    Args:
        news (pd.DataFrame) : The dataframe containing the data from the news csv.
        behaviors (pd.DataFrame) : The dataframe containing the data from the behaviors csv.

    Returns:
        None.
    """
    db_connection = create_connection()
    user_interaction_data = dpm.decompose_interactions(num_iterations=100000, news=news, behaviors=behaviors)
    user_interaction_data.to_sql('user_behaviors', if_exists='replace', con=db_connection, index=False)
    print("Push to Database Successful")
